In [7]:
pip install pandas ortools openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.26.1 which is incompatible.
tensorflow-metadata 1.13.1 requires absl-py<2.0.0,>=0.9, but you have absl-py 2.1.0 which is incompatible

In [20]:
import pandas as pd
import datetime
from ortools.sat.python import cp_model


class Employee:
    def __init__(self, name, skills, daily_hours):
        self.name = name
        self.skills = skills
        self.daily_hours = daily_hours  # Max hours they can work in a day
        self.remaining_hours = daily_hours

    def __str__(self):
        return f"{self.name} (Skills: {self.skills}, Remaining Hours: {self.remaining_hours})"


class Project:
    def __init__(self, title, required_skills, hours_needed, deadline):
        self.title = title
        self.required_skills = required_skills
        self.hours_needed = hours_needed
        self.deadline = deadline

    def __str__(self):
        return f"{self.title} (Required Skills: {self.required_skills}, Hours Needed: {self.hours_needed}, Deadline: {self.deadline})"


class TaskScheduler:
    def __init__(self):
        self.employees = []
        self.projects = []

    def add_employee(self, name, skills, daily_hours):
        self.employees.append(Employee(name, skills, daily_hours))

    def add_project(self, title, required_skills, hours_needed, deadline):
        self.projects.append(Project(title, required_skills, hours_needed, deadline))

    def schedule_tasks(self):
        # CSP Solver
        model = cp_model.CpModel()

        # Variables: Assign each project to an employee
        assignments = {}
        for project in self.projects:
            assignments[project.title] = [
                model.NewBoolVar(f"{project.title}_{employee.name}")
                for employee in self.employees
            ]

        # Constraints
        for i, project in enumerate(self.projects):
            # Each project must be assigned to exactly one employee
            model.Add(sum(assignments[project.title]) == 1)

            for j, employee in enumerate(self.employees):
                # Constraint: Employee must have required skills
                model.AddBoolOr([
                    assignments[project.title][j].Not()
                ] + [
                    skill.lower() in [s.lower() for s in employee.skills]
                    for skill in project.required_skills
                ])

                # Constraint: Employee must have enough remaining hours
                model.Add(
                    assignments[project.title][j] * project.hours_needed
                    <= employee.remaining_hours
                )

        # Objective: Minimize the total remaining hours across employees
        total_remaining_hours = sum(
            e.remaining_hours for e in self.employees
        )
        model.Minimize(total_remaining_hours)

        # Solve
        solver = cp_model.CpSolver()
        status = solver.Solve(model)

        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            for i, project in enumerate(self.projects):
                for j, employee in enumerate(self.employees):
                    if solver.Value(assignments[project.title][j]):
                        employee.remaining_hours -= project.hours_needed
                        print(
                            f"Project '{project.title}' assigned to Employee '{employee.name}'"
                        )
        else:
            print("No feasible solution found. Please check your constraints.")

def view_assignments(self):
    print("\nEmployee Assignments:")
    for employee in self.employees:
        print(employee)  # Will now display employee name, skills, remaining hours, and assigned projects

    print("\nProject Assignments:")
    for project in self.projects:
        # List assigned employees for each project
        assigned_employees = [
            employee.name for employee in self.employees if project in employee.assigned_projects
        ]
        print(f"{project.title}: Assigned to: {', '.join(assigned_employees) if assigned_employees else 'No employees assigned'}")


# Main function
def run_task_scheduler():
    scheduler = TaskScheduler()

    while True:
        print("\nTask Scheduler Menu")
        print("1. Add Employee")
        print("2. Add Project")
        print("3. Schedule Tasks")
        print("4. View Assignments")
        print("5. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            # Add employee
            name = input("Enter employee name: ")
            skills = input("Enter skills (comma-separated): ").split(',')
            daily_hours = int(input("Enter daily hours: "))
            scheduler.add_employee(name, skills, daily_hours)
        elif choice == "2":
            # Add project
            title = input("Enter project title: ")
            required_skills = input("Enter required skills (comma-separated): ").split(',')
            hours_needed = int(input("Enter hours needed: "))
            deadline = input("Enter deadline (YYYY-MM-DD): ")
            deadline_date = datetime.datetime.strptime(deadline, "%Y-%m-%d")
            scheduler.add_project(title, required_skills, hours_needed, deadline_date)
        elif choice == "3":
            # Schedule tasks
            scheduler.schedule_tasks()
        elif choice == "4":
            # View assignments
            scheduler.view_assignments()
        elif choice == "5":
            # Exit
            break
        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    run_task_scheduler()



Task Scheduler Menu
1. Add Employee
2. Add Project
3. Schedule Tasks
4. View Assignments
5. Exit
Enter your choice: 3

Task Scheduler Menu
1. Add Employee
2. Add Project
3. Schedule Tasks
4. View Assignments
5. Exit
Enter your choice: 1
Enter employee name: safi
Enter skills (comma-separated): SEO,SMM
Enter daily hours: 8

Task Scheduler Menu
1. Add Employee
2. Add Project
3. Schedule Tasks
4. View Assignments
5. Exit
Enter your choice: 2
Enter project title: Media
Enter required skills (comma-separated): SMM
Enter hours needed: 4
Enter deadline (YYYY-MM-DD): 2024-5-5

Task Scheduler Menu
1. Add Employee
2. Add Project
3. Schedule Tasks
4. View Assignments
5. Exit
Enter your choice: 3
Project 'Media' assigned to Employee 'safi'

Task Scheduler Menu
1. Add Employee
2. Add Project
3. Schedule Tasks
4. View Assignments
5. Exit
Enter your choice: 5
